In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random
from skimage import io, transform, exposure
from skimage.transform import ProjectiveTransform

In [2]:
training_data = {
    'feature': [],
    'label': []
}

base_dir = os.path.join(os.getcwd(), 'Training_Images')

In [3]:
for i, folder in enumerate(os.listdir(base_dir)):
    folder_path = os.path.join(base_dir, folder)
    for file in os.listdir(folder_path):
        if file.endswith('ppm'):
            file_path = os.path.join(folder_path, file)
            image = np.array(io.imread(file_path))
            image = transform.resize(image, (32, 32), mode='edge')
            training_data['feature'].append(image)
            training_data['label'].append(i)

In [ ]:
print(len(training_data['feature']), len(training_data['label']))

In [5]:
with open('trainingData.pkl', 'wb') as file:
    pickle.dump(training_data, file)

In [ ]:
classes, classIndex, classCount = np.unique(training_data['label'], return_index=True, return_counts='True')
plt.bar(classes, classCount)
plt.xlabel('Classification')
plt.ylabel('Number of Examples')
plt.title('Traffic Classifier Data')
plt.show()

In [2]:
df = pd.read_csv('trafficsignclassifier.csv')
with open('trainingData.pkl', 'rb') as file:
    pickled_data = pickle.load(file)

In [ ]:
print(len(pickled_data['feature']), len(pickled_data['label']))

In [ ]:

for c, index, count in zip(classes, classIndex, classCount):
    print(f'Class {c}: {df['signNames'][c]}')
    print(f'Samples: {count}')
    fig = plt.figure(figsize=(6, 1))
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0.05, wspace=0.05)
    randomSample = random.sample(range(index, index + count - 1), 10)
    for i in range(10):
        axis = fig.add_subplot(1, 10, i + 1, xticks=[], yticks=[])
        axis.imshow(training_data['feature'][randomSample[i]])
    plt.show()

In [4]:
#Image Transform/Augmentate Functions
def image_flip(feature, label=None):
    vert = [1, 5, 12, 15, 17]
    hori = [11, 12, 13, 15, 17, 18, 22, 26, 30, 35]
    both = [32, 40]
    chng = {19: 20, 20: 19,
            33: 34, 34: 33,
            36: 37, 37: 36,
            38: 39, 39: 38}

    if label in vert:
        vert_flipped = np.flip(feature, axis=0)
        yield vert_flipped, label
    
    if label in hori:
        hori_flipped = np.flip(feature, axis=1)
        yield hori_flipped, label
    
    if label in both:
        new_image = np.flip(feature, axis=(0, 1))
        yield new_image, label
    
    if label in chng:
        flipped_image = np.flip(feature, axis=0)
        yield flipped_image, chng[label]
    

def image_warp(feature, label=None, intensity=0.75):
    delta = 32 * .3 * intensity
    tl_top = random.uniform(-delta, delta)
    tl_left = random.uniform(-delta, delta)
    bl_bot = random.uniform(-delta, delta)
    bl_left = random.uniform(-delta, delta)
    tr_top = random.uniform(-delta, delta)
    tr_right = random.uniform(-delta, delta)
    br_bot = random.uniform(-delta, delta)
    br_right = random.uniform(-delta, delta)

    tf = ProjectiveTransform()
    tf.estimate(np.array(((tl_left, tl_top), 
                          (bl_left, 32 - bl_bot), 
                          (32 - br_right, 32 - br_bot), 
                          (32 - tr_right, tr_top))),
                np.array(((0, 0),
                          (0, 32),
                          (32, 32),
                          (32, 0)))
                )

    new_image = transform.warp(feature, tf, order=1, mode='edge')
    yield new_image, label

def image_rotate(feature, label=None, intensity=0.75):
    delta = 30 * intensity
    new_image = transform.rotate(feature, angle=(random.uniform(-delta, delta)), mode='edge')
    yield new_image, label

def grayscale_contrast(feature, label=None):
    new_image = 0.2989 * feature[:, :, 0] + 0.5870 * feature[:, :, 1] + 0.1140 * feature[:, :, 2]
    new_image = exposure.equalize_adapthist(new_image)
    return new_image, label

Functions

In [ ]:

img = pickled_data['feature'][random.randint(0, len(pickled_data['feature']))]
rot, _ = image_rotate(img)
gray, _ = grayscale_contrast(img)
rot_gray, _ = grayscale_contrast(rot)
flipped = np.flip(img, axis=0)
flipped1 = np.flip(img, axis=1)
flipped2 = np.flip(img, axis=(0, 1))

fig, axs = plt.subplots(1, 7, figsize=(5, 5))
axs[0].imshow(img)
axs[0].set_xticks([])
axs[0].set_yticks([])
axs[1].imshow(rot)
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[2].imshow(gray, cmap=plt.cm.gray)
axs[2].set_xticks([])
axs[2].set_yticks([])
axs[3].imshow(rot_gray, cmap=plt.cm.gray)
axs[3].set_xticks([])
axs[3].set_yticks([])
axs[4].imshow(flipped)
axs[4].set_xticks([])
axs[4].set_yticks([])
axs[5].imshow(flipped1)
axs[5].set_xticks([])
axs[5].set_yticks([])
axs[6].imshow(flipped2)
axs[6].set_xticks([])
axs[6].set_yticks([])
plt.show()


In [ ]:
feature, label = pickled_data['feature'][:], pickled_data['label'][:]

for feat, labl in zip(feature[:], label[:]):
    for f, l in image_flip(feat, labl):
        feature.append(f)
        label.append(l)

print('Done Filp')

for feat, labl in zip(feature[:], label[:]):
    for f, l in image_rotate(feat, labl):
        feature.append(f)
        label.append(l)

print('Done Rotate')

for feat, labl in zip(feature[:], label[:]):
    for f, l in image_warp(feat, labl):
        feature.append(f)
        label.append(l)

print('Done Warp')

print(len(feature), len(label))


Unbalanced Dataset

In [ ]:
classes, classIndex, classCount = np.unique(label, return_index=True, return_counts='True')
plt.bar(classes, classCount)
plt.xlabel('Classification')
plt.ylabel('Number of Examples')
plt.title('Traffic Classifier Data')
plt.show()

In [ ]:
classes, classIndex, classCount = np.unique(pickled_data['label'], return_index=True, return_counts='True')
plt.bar(classes, classCount)
plt.xlabel('Classification')
plt.ylabel('Number of Examples')
plt.title('Traffic Classifier Data')
plt.show()